In [3]:
import chemistry_psql as cp
import importlib
importlib.reload(cp)
import psycopg2
import pandas as pd

dfm = pd.read_csv("../Nodes_Seb.csv")
#print(dfm)

rbp = dfm[['name']]
rbp_list = rbp.to_records(index = False)
#print(rbp_list)

#imp_struct = ()

#for i in struct_list:
    #print(i)
#    cp.SmilesInsert(i)


print()
print('# # # Target Search # # #')
print()

rbp_chembl =[]

db_chem = psycopg2.connect(host = "localhost", dbname="chembl_28", user="postgres", password="postgres")
cur = db_chem.cursor()
target_num = 0
rbp_total = 0
rbp_found = 0
for i in rbp_list:
    j = i[0]
    #print()
    #print(i,j)
    sql = 'select cs.component_synonym, td.chembl_id as "CHEMBL_Target", td.pref_name \
    from public.component_synonyms cs \
    join public.component_sequences cl on cl.component_id = cs.component_id \
    join public.target_components tc on cl.component_id = tc.component_id \
    join public.target_dictionary td on td.tid = tc.tid \
    where upper(cs.component_synonym)=\''+j+'\' \
    and td.target_type = \'SINGLE PROTEIN\';'
    #print(sql)
    cur.execute(sql)
    targetlist = cur.fetchall()
    #print(targetlist, len(targetlist))
    rbp_total = rbp_total + 1
    if len(targetlist)>0:
        rbp_chembl.append(targetlist)
        target_num = target_num + len(targetlist)
        rbp_found = rbp_found + 1

print()
print(rbp_total, rbp_found, target_num)
print()
#print(rbp_chembl)



#test = ("C1CCCCOCCCC1","C1CCCCNCCCC1","C1CCCCSCCCC1")

#cp.SmilesInsert(test)


# # # Target Search # # #


731 200 282

[('PTBP1', 'CHEMBL1293230', 'Polypyrimidine tract-binding protein 1')]
[('SF1', 'CHEMBL4666', 'Steroidogenic factor 1')]
[('EWSR1', 'CHEMBL3351202', 'RNA-binding protein EWS')]
[('SRPK2', 'CHEMBL5668', 'Serine/threonine-protein kinase SRPK2')]
[('ELAVL3', 'CHEMBL4105924', 'ELAV-like protein 3')]
[('DUSP14', 'CHEMBL1764941', 'Dual specificity protein phosphatase 14')]
[('CFL1', 'CHEMBL1075129', 'Cofilin-1')]
[('actb', 'CHEMBL5170', 'beta-actin'), ('ACTB', 'CHEMBL2062353', 'Actin, cytoplasmic 1')]
[('PCBP2', 'CHEMBL4295826', 'Poly(rC)-binding protein 2')]
[('DAPK1', 'CHEMBL2558', 'Death-associated protein kinase 1')]
[('CLK2', 'CHEMBL4225', 'Dual specificity protein kinase CLK2'), ('Clk2', 'CHEMBL1075281', 'Dual specificity protein kinase CLK2')]
[('VIM', 'CHEMBL3712854', 'Vimentin')]
[('RPS6', 'CHEMBL3351215', '40S ribosomal protein S6')]
[('RNMT', 'CHEMBL4295662', 'mRNA cap guanine-N7 methyltransferase')]
[('SF3B2', 'CHEMBL1229011', 'Splicing f

In [37]:
chemex = []
rbp_chemlist = []

for i in rbp_chembl:
    #print(i)
    for j in i:
        #print(j, j[1])
        if j[1] not in chemex:
            chemex.append(j[1])
            rbp_chemlist.append(j)

chemtup = tuple(chemex)
#print(len(chemex),chemtup)
#print(rbp_chemlist)
print()
df_rbp = pd.DataFrame(rbp_chemlist, columns =['Name', 'ChemBL_ID', 'pref_name'])
print(df_rbp)

print(rbp)
rbpl = rbp['name'].to_list()
print(rbpl)
rbpt = tuple(rbpl)
print(rbpt)


       Name      ChemBL_ID                                 pref_name
0     PTBP1  CHEMBL1293230    Polypyrimidine tract-binding protein 1
1       SF1     CHEMBL4666                    Steroidogenic factor 1
2     EWSR1  CHEMBL3351202                   RNA-binding protein EWS
3     SRPK2     CHEMBL5668     Serine/threonine-protein kinase SRPK2
4    ELAVL3  CHEMBL4105924                       ELAV-like protein 3
..      ...            ...                                       ...
264    MDH2     CHEMBL3444        Malate dehydrogenase mitochondrial
265    MDH2     CHEMBL5917       Malate dehydrogenase, mitochondrial
266    Mdh2  CHEMBL2176835       Malate dehydrogenase, mitochondrial
267   CNOT4  CHEMBL4105770  CCR4-NOT transcription complex subunit 4
268  RPS27A  CHEMBL4523597      Ubiquitin-40S ribosomal protein S27a

[269 rows x 3 columns]
         name
0       CTBP1
1       RBM14
2       CPSF6
3       PTBP1
4       PTBP2
..        ...
726      NOL7
727  RPS19BP1
728      PUS7
729    

In [39]:
sql = 'select distinct cs.component_synonym, cmd.chembl_id, cms.synonyms, cms.syn_type, cmd.molecule_type, \
cmd.max_phase, ccs.canonical_smiles, td.chembl_id as "CHEMBL_Target", td.pref_name, cas.assay_id, cas.chembl_id \
as "ChemBL_Assay", cas.description, caa.standard_value, caa.standard_units \
from public.component_synonyms cs \
join public.component_sequences cl on cl.component_id = cs.component_id \
join public.target_components tc on cl.component_id = tc.component_id \
join public.target_dictionary td on td.tid = tc.tid \
join public.assays cas on cas.tid = td.tid \
join public.activities caa on caa.assay_id = cas.assay_id \
join public.compound_structures ccs on caa.molregno = ccs.molregno \
join public.molecule_dictionary cmd on caa.molregno = cmd.molregno \
left join public.molecule_synonyms cms on cms.molregno = cmd.molregno \
where td.chembl_id in '+ str(chemtup) +' \
and upper(cs.component_synonym) in '+str(rbpt)+' \
and cas.assay_type = \'B\' \
order by cs.component_synonym;'
print(sql)
#print(sql2)


print()
#db_chem = psycopg2.connect(host = "localhost", dbname="chembl_28", user="postgres", password="postgres")
#cur = db_chem.cursor()
#cur.execute(sql)
#mollist = cur.fetchall()

#print(mollist)

select distinct cs.component_synonym, cmd.chembl_id, cms.synonyms, cms.syn_type, cmd.molecule_type, cmd.max_phase, ccs.canonical_smiles, td.chembl_id as "CHEMBL_Target", td.pref_name, cas.assay_id, cas.chembl_id as "ChemBL_Assay", cas.description, caa.standard_value, caa.standard_units from public.component_synonyms cs join public.component_sequences cl on cl.component_id = cs.component_id join public.target_components tc on cl.component_id = tc.component_id join public.target_dictionary td on td.tid = tc.tid join public.assays cas on cas.tid = td.tid join public.activities caa on caa.assay_id = cas.assay_id join public.compound_structures ccs on caa.molregno = ccs.molregno join public.molecule_dictionary cmd on caa.molregno = cmd.molregno left join public.molecule_synonyms cms on cms.molregno = cmd.molregno where td.chembl_id in ('CHEMBL1293230', 'CHEMBL4666', 'CHEMBL3351202', 'CHEMBL5668', 'CHEMBL4105924', 'CHEMBL1764941', 'CHEMBL1075129', 'CHEMBL5170', 'CHEMBL2062353', 'CHEMBL429582